## Hierarchical modularity in multiple cortical areas

Analysis of the data by [Stringer et al. 2019](science.org/doi/10.1126/science.aav7893) 

https://github.com/MouseLand/stringer-pachitariu-et-al-2018a/tree/master

https://janelia.figshare.com/articles/dataset/Eight-probe_Neuropixels_recordings_during_spontaneous_behaviors/7739750/4

Eight-probe Neuropixels recordings in three mice of spontaneous activity.

Each "spks" is a structure of length 8, where each entry is a different probe (these probes were recorded simultaneously). It contains the spike times and the cluster identity of each spike.
```
load(fullfile(ephysroot, sprintf('spks%s_Feb18.mat' , mouse_name)));
spks(k).st = spike times (in seconds)
spks(k).clu = cluster identity of each spike in st (which neuron does spike belong to)
spks(k).Wheights = height of each cluster on the probe
```

Information about each probe's location is in "probeLocations.mat". The location of each site on the probe in microns in the Allen CCF framework is given in "ccfCoords". The brain area for each site is in "borders" as a function of the height of the site. To view the probe inside a wire-mesh brain, download and add to your path this toolbox: https://github.com/cortex-lab/allenCCF . You will also need to add to your path the npy-matlab toolbox: https://github.com/kwikteam/npy-matlab .

The behavioral file is the processed version of a mouse face movie (time x pixels x pixels). For raw videos, see this:

https://figshare.com/articles/Behavioral_videos_for_8-Neuropixels_recordings_from_Stringer_et_al_2019_Science/8378360

We processed these by taking the singular value decomposition of it.
```
beh = load(fullfile(ephysroot, sprintf('%s_face_proc.mat', mouse_name)));
beh.motionSVD = timepoints x components (left singular vectors * singular values)
beh.motionMask = pixels x pixels x components (right singular vectors)
beh.times = times of behavioral frames (in seconds) in same timeframe as spikes
```
See the "ephysLoad.m" script for how to load each into matlab and better understand the structures.


In [58]:
from platform import python_version
print(python_version())

from builtins import exec
exec(open("./imports_functions.py").read())

%matplotlib inline

3.10.4


In [2]:
if not os.path.exists("stringer/7739750.zip"):
    print("Downloading RSC data ...")
    resp = wget.download("https://janelia.figshare.com/ndownloader/articles/7739750/versions/4", "stringer/7739750.zip")
    print("... Done: "+resp)



KeyboardInterrupt



base this analysis on the file `ephysLoad.m`

In [78]:
from scipy import io

probeLoc = io.loadmat('stringer/7739750/probeLocations.mat')
probeBorders = io.loadmat('stringer/7739750/probeBorders.mat', squeeze_me=True)

mouse_names = ['Krebs','Waksman','Robbins']
# start of spontaneous activity in each mouse (in seconds)
tstart = [3811, 3633, 3323]

for imouse in range(len(mouse_names)):
    print(mouse_names[imouse])
    
    spks = io.loadmat('stringer/7739750/spks/spks%s_Feb18.mat'%mouse_names[imouse], squeeze_me=True)
    faces = io.loadmat('stringer/7739750/faces/%s_face_proc.mat'%mouse_names[imouse], squeeze_me=True)

    # probe k
    # for k in range(8):
    k = 7
    
    # spike times (in seconds)
    st = spks['spks'][k][0]
    print(len(st))
    # clusters
    clu = spks['spks'][k][1]
    print("clusters (cells) of the spikes",len(np.unique(clu)))
    # cluster heights (in microns)
    # (see siteCoords to convert to site location)
    Wh = spks['spks'][k][2]
    
    # select by height to get only cortex

    # processed faces, to correlate behavior
    motSVD = faces['motionSVD']
    video_timestamps = faces['times']

    # where is the probe in the brain (consolidated labels)
    # borders are in microns
    # use Wh to determine which clusters are in which brain region
    borders = probeBorders['probeBorders'][imouse]['borders'][k]
    for j in range(len(borders)):
       b = borders[j]
       print('upper border %d um, lower border %d um, area %s'%(b[0],b[1],b[2]))
       wneurons = np.logical_and(Wh>=b[1], Wh<b[0])
       nn = wneurons.sum()
       print('%d neurons in %s'%(nn,b[-1]))

    # # where is the probe in the brain (in microns)
    # ccfCoords = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['ccfCoords']
    # # name of area in Allen ontology by site on electrode
    # ccfNames = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['ccfOntology']
    # # coordinates of each site on the electrode
    # siteCoords = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['siteCoords']

    print()

Krebs
1765806
165
upper border 4000 um, lower border 2600 um, area V1
64 neurons in V1
upper border 2600 um, lower border 1500 um, area HPF
46 neurons in HPF
upper border 1500 um, lower border 0 um, area TH
55 neurons in TH

Waksman
4716219
249
upper border 4000 um, lower border 2850 um, area V1
96 neurons in V1
upper border 2850 um, lower border 1650 um, area HPF
73 neurons in HPF
upper border 1650 um, lower border 0 um, area TH
80 neurons in TH

Robbins
5362988
305
upper border 4000 um, lower border 2600 um, area HPF
33 neurons in HPF
upper border 2600 um, lower border 0 um, area TH
272 neurons in TH

